This Notebook contains my process through creating my first algorithmic trading bot, I guess its my "Hello World" for this project. So during this notebook I probably have no clue what I'm doing, but I hope theres clear progress from this notebook to future entries in the project. My favorite ticker to look at is the NQ future, so thats what I'll do the project on.

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go

### Importing Data
Getting data from yahoo finance data api

In [108]:
nqdf = yf.download('NQ=F', start='2025-02-4', end='2025-04-1', interval='15m')
nqdf.columns = nqdf.columns.get_level_values(0)

[*********************100%***********************]  1 of 1 completed


In [109]:
nqdf

Price,Close,High,Low,Open,Volume
Datetime,,,,,
2025-02-04 05:00:00+00:00,21306.00,21348.50,21289.00,21301.50,1470
2025-02-04 05:15:00+00:00,21334.50,21344.25,21242.50,21302.50,4369
2025-02-04 05:30:00+00:00,21352.50,21361.50,21317.75,21333.50,2149
2025-02-04 05:45:00+00:00,21358.25,21410.00,21344.00,21354.25,2405
2025-02-04 06:00:00+00:00,21359.75,21374.75,21337.50,21359.25,1395
...,...,...,...,...,...
2025-04-01 02:45:00+00:00,19380.00,19381.75,19364.75,19366.25,939
2025-04-01 03:00:00+00:00,19378.25,19386.75,19373.50,19380.25,778
2025-04-01 03:15:00+00:00,19371.50,19379.25,19367.50,19379.25,665


Signal: Engulfing Candle
This is probably the most basic signal there is. Its simply when the following candle opens and closes below the previous (for a bearish indicator, opposite for bullish)  
<img src="pictures/engulfingpic.jpg" alt="Description" style="display:block;margin:auto" width="300" height="200"/>


In [110]:
def engulf_signal(df):
    """
    Generate buy/sell signals based on the closing prices of the candles. If the 2nd candle in a sequence closes below or above
    the opening price of the candle before, the signal will occur.
    For a sell signal, the closing price of the 2nd candle must be below the opening price of the previous candle and the function will return 1.
    For a buy signal, the closing price of the 2nd candle must be above the opening price of the previous candle and the function will return 2.
    No signal returns 0
    """
    open_ = float(df.Open.iloc[-1])
    close_ = float(df.Close.iloc[-1])
    prev_open = float(df.Open.iloc[-2])
    prev_close = float(df.Close.iloc[-2])

    #Sell signal
    if (open_>close_ and
        prev_open<prev_close and
        close_<prev_open and
        open_>=prev_close):
        return 1

    #Buy signal
    elif (open_<close_ and
          prev_open>prev_close and
          close_>prev_open and
          open_<=prev_close):
        return 2
    
    #No signal
    else:
        return 0

In [111]:
#Checking through our df for signals
signal = []
signal.append(0)
for i in range(1, len(nqdf)):
    data = nqdf.iloc[i-1:i+1]
    signal.append(engulf_signal(data))
nqdf['Signal'] = signal

In [112]:
nqdf.Signal.value_counts()

Signal
0    3078
1     252
2     229
Name: count, dtype: int64

In [113]:
nqdf.Close.iloc[0]

np.float64(21306.0)

Plotting the signals on a chart

In [125]:
def plot_candlestick_signals(df, day=None, start_time=None, end_time=None):
    """
    Plot candlestick chart with engulfing buy/sell signals for a specific day and optional time range.
    
    Parameters:
        df (DataFrame): OHLC dataframe with a datetime index and a 'Signal' column.
        day (str or datetime.date, optional): The date to plot (e.g., '2025-03-10').
        start_time (str, optional): Start time (e.g., '09:30') in HH:MM format.
        end_time (str, optional): End time (e.g., '16:00') in HH:MM format.
    """
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    df = df.dropna(subset=['Open', 'High', 'Low', 'Close'])

    # Filter by day
    if day is not None:
        day = pd.to_datetime(day).date()
        df = df[df.index.date == day]

    # Filter by time range
    if start_time or end_time:
        # Convert to datetime.time objects
        if start_time:
            start_time = pd.to_datetime(start_time).time()
        if end_time:
            end_time = pd.to_datetime(end_time).time()

        df = df[(df.index.time >= (start_time or df.index.time.min())) &
                (df.index.time <= (end_time or df.index.time.max()))]

    if df.empty:
        print("No data for the specified day/time range.")
        return

    # Plot
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candles'
    ))

    # Sell signals
    sell_signals = df[df['Signal'] == 1]
    fig.add_trace(go.Scatter(
        x=sell_signals.index,
        y=sell_signals['High'],
        mode='markers',
        marker=dict(color='red', symbol='triangle-down', size=10),
        name='Sell Signal'
    ))

    # Buy signals
    buy_signals = df[df['Signal'] == 2]
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['Low'],
        mode='markers',
        marker=dict(color='green', symbol='triangle-up', size=10),
        name='Buy Signal'
    ))

    fig.update_layout(
        title=f'NQ=F Candlestick Chart with Signals ({day})',
        xaxis_title='Time',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        height=700
    )

    fig.show()


In [130]:
plot_candlestick_signals(nqdf, day='2025-03-10')

This signal obviously should not be used by itself but it definitely can lead to some profit..